In [324]:
import pandas as pd
import os
import json
import numpy as np

In [325]:
no1 = "No.1"
no1 = no1.center(100,"=")
print(f"\n",no1)


 ================================================No.1================================================


In [326]:
# create a function to load all the JSON files inside a folder and normalize them then append the files into a data_list
def load_and_normalize(folder_path):
    data_list = []

    # loop into the folder, scanning all the files inside the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        with open(file_path,"r") as f:
            data = json.load(f)
            data_list.append(data)
            
    # normalize the datas inside data_list and update the data_list directly
    data_list = pd.json_normalize(data_list)
    
    # sort the datas inside data_list ny id and ts(timestamp)
    data_list = data_list.sort_values(by=["id","ts"])
    
    return data_list

In [327]:
data_folder = os.environ.get("DATA_FOLDER", "./data")
accounts_path = os.path.join(data_folder,"accounts")
cards_path = os.path.join(data_folder,"cards")
savings_accounts_path = os.path.join(data_folder,"savings_accounts")

In [328]:
# Feeling dataframe "df_accounts" with all the JSON data files located in "./data/accounts" using load_and_normalize function
df_accounts = load_and_normalize("./data/accounts")

# renaming and combining the features/columns
df_accounts["timestamp"] = df_accounts["ts"]
df_accounts["account_id"] = df_accounts["data.account_id"]
df_accounts["name"] = df_accounts["data.name"]
df_accounts["email"] = df_accounts["set.email"].combine_first(df_accounts["data.email"])
df_accounts["savings_account_id"] = df_accounts["set.savings_account_id"]
df_accounts["card_id"] = df_accounts["set.card_id"]

# combine only if there is a "set" column where it represents an update on the data with the same "name"
# combining column "set.address" with "data.address" into a new column named "address"
df_accounts["address"] = df_accounts["set.address"].combine_first(df_accounts["data.address"])

# combining column "set.phone_number" with "data.phone_number" into a new column named "phone_number"
df_accounts["phone_number"] = df_accounts["set.phone_number"].combine_first(df_accounts["data.phone_number"])

# choosing the needed columns that"s been cleaned
cols = ["id","timestamp","account_id","name","address","phone_number","email","savings_account_id","card_id"]
df_accounts = df_accounts[cols]

# grouping all the datas by using "id" as reference and feeling the empty data with the the ones above it (updating from the previous data
df_accounts = df_accounts.groupby("id").ffill()

print("\nACCOUNTS:")
print(df_accounts.to_string())


ACCOUNTS:
       timestamp account_id     name   address phone_number                    email savings_account_id card_id
0  1577863800000         a1  Anthony  New York     12345678     anthony@somebank.com                NaN     NaN
1  1577865600000         a1  Anthony  New York     87654321     anthony@somebank.com                NaN     NaN
2  1577890800000         a1  Anthony  New York     87654321     anthony@somebank.com                sa1     NaN
3  1577894400000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1     NaN
4  1577926800000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1      c1
5  1579078860000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1        
6  1579163400000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1      c2


In [329]:
# Feeling dataframe "df_accounts" with all the JSON data files located in "./data/cards" using load_and_normalize function
df_cards = load_and_normalize("./data/cards")

# renaming and combining the features/columns
df_cards["timestamp"] = df_cards["ts"]
df_cards["card_id"] = df_cards["data.card_id"]
df_cards["card_number"] = df_cards["data.card_number"]
df_cards["monthly_limit"] = df_cards["data.monthly_limit"]

# combine only if there is a "set" column where it represents an update on the data with the same "name"
# combining column "set.credit_used" with "data.credit_used" into a new column named "credit_used"
df_cards["credit_used"] = df_cards["set.credit_used"].combine_first(df_cards["data.credit_used"])

# combining column "set.status" with "data.status" into a new column named "status"
df_cards["status"] = df_cards["set.status"].combine_first(df_cards["data.status"])

# choosing the needed columns that"s been cleaned
cols = ["id","timestamp","card_id","card_number","credit_used","monthly_limit","status"]
df_cards = df_cards[cols]

# grouping all the datas by using "id" as reference and feeling the empty data with the the ones above it (updating from the previous data
df_cards = df_cards.groupby("id").ffill()

print("\nCARDS:")
print(df_cards.to_string())


CARDS:
       timestamp card_id card_number  credit_used  monthly_limit   status
0  1577926800000      c1    11112222          0.0        30000.0  PENDING
1  1578159000000      c1    11112222          0.0        30000.0   ACTIVE
2  1578313800000      c1    11112222      12000.0        30000.0   ACTIVE
3  1578420000000      c1    11112222      19000.0        30000.0   ACTIVE
4  1578654000000      c1    11112222          0.0        30000.0   ACTIVE
5  1579078800000      c1    11112222          0.0        30000.0   CLOSED
6  1579163400000      c2    12123434          0.0        70000.0  PENDING
7  1579298400000      c2    12123434          0.0        70000.0   ACTIVE
8  1579361400000      c2    12123434      37000.0        70000.0   ACTIVE


In [330]:
# Feeling dataframe "df_accounts" with all the JSON data files located in "./data/cards" using load_and_normalize function
df_saving_accounts = load_and_normalize("./data/savings_accounts")

# renaming and combining the features/columns
df_saving_accounts["timestamp"] = df_saving_accounts["ts"]
df_saving_accounts["savings_account_id"] = df_saving_accounts["data.savings_account_id"]
df_saving_accounts["status"] = df_saving_accounts["data.status"]

# combine only if there is a "set" column where it represents an update on the data with the same "name"
# combining column "set.balance" with "data.balance" into a new column named "balance"
df_saving_accounts["balance"] = df_saving_accounts["set.balance"].combine_first(df_saving_accounts["data.balance"])

# combining column "set.interest_rate_percent" with "data.interest_rate_percent" into a new column named "interest_rate_percent"
df_saving_accounts["interest_rate_percent"] = df_saving_accounts["set.interest_rate_percent"].combine_first(df_saving_accounts["data.interest_rate_percent"])

# choosing the needed columns that"s been cleaned
cols = ["id","timestamp","savings_account_id","balance","interest_rate_percent","status"]
df_saving_accounts = df_saving_accounts[cols]

# grouping all the datas by using "id" as reference and feeling the empty data with the the ones above it (updating from the previous data)
df_saving_accounts = df_saving_accounts.groupby("id").ffill()

print("\nSAVING ACCOUNTS:")
print(df_saving_accounts.to_string())


SAVING ACCOUNTS:
       timestamp savings_account_id  balance  interest_rate_percent  status
0  1577890800000                sa1      0.0                    1.5  ACTIVE
1  1577955600000                sa1  15000.0                    1.5  ACTIVE
2  1578159060000                sa1  15000.0                    3.0  ACTIVE
3  1578648600000                sa1  40000.0                    3.0  ACTIVE
4  1578654000000                sa1  21000.0                    3.0  ACTIVE
5  1579078860000                sa1  21000.0                    1.5  ACTIVE
6  1579298460000                sa1  21000.0                    4.0  ACTIVE
7  1579505400000                sa1  33000.0                    4.0  ACTIVE


In [331]:
no2 = "No.2"
no2 = no2.center(100,"=")
print(f"\n",no2)


 ================================================No.2================================================


In [332]:
# creating a dataframe (account_card_join_df) to store a join between df_accounts and df_cards with card_id as reference
# using outer join since it will merged the same data by following the reference and filling nan if it's not.
account_card_join_df = pd.merge(df_cards,df_accounts,on=["card_id","timestamp"],how="outer")

cols = ["timestamp","account_id","name","address","phone_number","email","savings_account_id","card_id","card_number","credit_used","monthly_limit","status"]

# sort the table by timestamp
account_card_join_df = account_card_join_df[cols].sort_values("timestamp")

# filling nan value with the data above
account_card_join_df = account_card_join_df.fillna(method="ffill")
print(f"\nMerged Table Account and Cards: \n",account_card_join_df.to_string())


Merged Table Account and Cards: 
         timestamp account_id     name   address phone_number                    email savings_account_id card_id card_number  credit_used  monthly_limit   status
9   1577863800000         a1  Anthony  New York     12345678     anthony@somebank.com                NaN     NaN         NaN          NaN            NaN      NaN
10  1577865600000         a1  Anthony  New York     87654321     anthony@somebank.com                NaN     NaN         NaN          NaN            NaN      NaN
11  1577890800000         a1  Anthony  New York     87654321     anthony@somebank.com                sa1     NaN         NaN          NaN            NaN      NaN
12  1577894400000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1     NaN         NaN          NaN            NaN      NaN
0   1577926800000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222          0.0        30000.0  P

In [333]:
final_merged_df = pd.merge(df_saving_accounts,account_card_join_df,on=["savings_account_id","timestamp","status"],how="outer")
cols = ["timestamp","account_id","name","address","phone_number","email","savings_account_id","card_id","card_number","monthly_limit","status","credit_used","balance","interest_rate_percent"]
final_merged_df = final_merged_df.sort_values(["timestamp"])
final_merged_df = final_merged_df.fillna(method="ffill")

print(f"\nFinal Table: \n\n",final_merged_df[cols].to_string())


Final Table: 

         timestamp account_id     name   address phone_number                    email savings_account_id card_id card_number  monthly_limit   status  credit_used  balance  interest_rate_percent
8   1577863800000         a1  Anthony  New York     12345678     anthony@somebank.com                NaN     NaN         NaN            NaN      NaN          NaN      NaN                    NaN
9   1577865600000         a1  Anthony  New York     87654321     anthony@somebank.com                NaN     NaN         NaN            NaN      NaN          NaN      NaN                    NaN
0   1577890800000         a1  Anthony  New York     87654321     anthony@somebank.com                sa1     NaN         NaN            NaN   ACTIVE          NaN      0.0                    1.5
10  1577890800000         a1  Anthony  New York     87654321     anthony@somebank.com                sa1     NaN         NaN            NaN   ACTIVE          NaN      0.0                    1.5
11  157789440

In [334]:
no3 = "No.3"
no3 = no3.center(100,"=")
print(f"\n",no3)


 ================================================No.3================================================


In [335]:
# dropping alla the rows that contains nan whicch are not needed to count the total transactions
final_merged_df = final_merged_df.dropna()

# filtering to only show the data that is from card_id c1
c1_df = final_merged_df.where(final_merged_df["card_id"] == "c1").dropna()

# defining if the data is a transaction by checking if the balance's been changed and credit's been used
c1_df["balance_changed"] = c1_df.groupby("card_id")["balance"].transform(lambda x: x.shift() !=x)
c1_df["credit_changed"] = c1_df.groupby("card_id")["credit_used"].transform(lambda x: x.shift() !=x)
c1_df["is_transaction"] = c1_df["balance_changed"] | c1_df["credit_changed"]

# count the total of transactions and store it in total_transaction_c1
total_transaction_c1 = c1_df["is_transaction"].sum() -1

cols = ["timestamp","account_id","name","address","phone_number","email","savings_account_id","card_id","card_number","monthly_limit","status","credit_used","balance"]

print(f"\nHistorical Table of Anthoy with card_id c1: \n\n",c1_df[cols].to_string())
print(f"Total transaction for card c1: ",total_transaction_c1)




Historical Table of Anthoy with card_id c1: 

        timestamp account_id     name  address phone_number                    email savings_account_id card_id card_number  monthly_limit   status  credit_used  balance
12  1.577927e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222        30000.0  PENDING          0.0      0.0
1   1.577956e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222        30000.0   ACTIVE          0.0  15000.0
13  1.578159e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222        30000.0   ACTIVE          0.0  15000.0
2   1.578159e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222        30000.0   ACTIVE          0.0  15000.0
14  1.578314e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1     

In [336]:
c2_df = final_merged_df.where(final_merged_df["card_id"] == "c2").dropna()

c2_df["balance_changed"] = c2_df.groupby("card_id")["balance"].transform(lambda x: x.shift() !=x)
c2_df["credit_changed"] = c2_df.groupby("card_id")["credit_used"].transform(lambda x: x.shift() !=x)
c2_df["is_transaction"] = c2_df["balance_changed"] | c2_df["credit_changed"]

total_transaction_c2 = c2_df["is_transaction"].sum() -1

print(f"\nHistorical Table of Anthoy with card_id c2: \n\n",c2_df[cols].to_string())
print(f"Total transaction for card c2: ",total_transaction_c2)


Historical Table of Anthoy with card_id c2: 

        timestamp account_id     name  address phone_number                    email savings_account_id card_id card_number  monthly_limit   status  credit_used  balance
18  1.579163e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c2    12123434        70000.0  PENDING          0.0  21000.0
19  1.579298e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c2    12123434        70000.0   ACTIVE          0.0  21000.0
6   1.579298e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c2    12123434        70000.0   ACTIVE          0.0  21000.0
20  1.579361e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c2    12123434        70000.0   ACTIVE      37000.0  21000.0
7   1.579505e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1     